In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [23]:
dpath = '/Users/baidu/anaconda3/tianchi/'
part1 = pd.read_table(dpath + 'meinian_round1_data_part1_20180408.txt', sep='$')
part2 = pd.read_table(dpath + 'meinian_round1_data_part2_20180408.txt', sep='$')
test = pd.read_csv(dpath + 'meinian_round1_test_a_20180408.csv', encoding='gb2312')
train = pd.read_csv(dpath + 'meinian_round1_train_20180408.csv', encoding='gb2312')


In [24]:
data = pd.concat([part1, part2], axis=0) #合并part1,part2
data.head()


,vid,table_id,field_results
0,002d1e4859fafd9ded2a2e1e7c839b62,2403,72.9
1,002d1e4859fafd9ded2a2e1e7c839b62,2404,166.5
2,002d1e4859fafd9ded2a2e1e7c839b62,2405,26.30
3,002d1e4859fafd9ded2a2e1e7c839b62,2420,79
4,002d1e4859fafd9ded2a2e1e7c839b62,0702,无


In [25]:
#不是所有原始用户都被标注
vids = data['vid'].unique()
train_vids = train['vid'].unique()
len(vids),len(train_vids)

(57298, 38199)

In [26]:
#只选择已在train数据集中标注了的用户
data = data[data['vid'].isin(list(train_vids))]
vids = data['vid'].unique()
len(vids),len(train_vids)


(38199, 38199)

In [27]:
train.head()


,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
0,002d1e4859fafd9ded2a2e1e7c839b62,165,100,2.08,1.29,3.24
1,92dd479df5e30ab6a0a1cf85ac53efc3,141,97,2.64,1.36,4.75
2,6bb59d517c4c70f8f50844d24fbd0355,120,80,1.37,1.25,2.66
3,0ebb42adae512906f7e1135da734ea63,100,70,1.27,2.21,1.73
4,ebe7811e919109c42c092abbd98b4ca6,110,80,0.8,1.87,2.21


In [28]:
#去掉vid,table_id重复行
data = data.drop_duplicates(['vid', 'table_id'], keep='first')

In [29]:
#依据索引为vid,特征为table_id,值为field_results重新整合
data_pivot = data.pivot(index='vid', columns='table_id', values='field_results')
data_pivot.shape

(38199, 2605)

In [30]:
#计算每个特征中空值的比例
count = data_pivot.count()   #每个特征的非空值计数
total = len(data_pivot)      #总的用户数
na_stat = count.map(lambda x : 1.0 - x/total).round(2)


In [31]:
#选取空值比例不大于x%的特征
tmp = na_stat[na_stat < 0.80]
keys = tmp.keys().tolist()
data_pivot = data_pivot[keys]

In [32]:
data_pivot.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 38199 entries, 000330ad1f424114719b7525f400660b to fffda084a188b379e403f64e62c2adf5
Data columns (total 201 columns):
0101      31127 non-null object
0102      37639 non-null object
0113      37311 non-null object
0114      37304 non-null object
0115      33977 non-null object
0116      37222 non-null object
0117      36589 non-null object
0118      36588 non-null object
0119      20904 non-null object
0120      18877 non-null object
0121      12977 non-null object
0122      12978 non-null object
0123      12978 non-null object
0124      14132 non-null object
0201      21674 non-null object
0202      21959 non-null object
0203      21932 non-null object
0206      18884 non-null object
0207      21011 non-null object
0208      21200 non-null object
0209      20113 non-null object
0210      20842 non-null object
0212      20770 non-null object
0215      20150 non-null object
0216      20532 non-null object
0217      21953 non-null object
0222 

In [33]:
#重置一下索引
data_pivot.reset_index(inplace=True)

In [34]:
#合并标注特征
data_pivot = data_pivot.sort_values(by=['vid'])
train = train.sort_values(by=['vid'])
for key in train.columns:
    if key != 'vid':
        data_pivot[key] = train[key]

In [35]:
data_pivot.columns[:50]
# 我来处理

Index(['vid', '0101', '0102', '0113', '0114', '0115', '0116', '0117', '0118',
       '0119', '0120', '0121', '0122', '0123', '0124', '0201', '0202', '0203',
       '0206', '0207', '0208', '0209', '0210', '0212', '0215', '0216', '0217',
       '0222', '0403', '0405', '0406', '0407', '0409', '0413', '0420', '0421',
       '0422', '0423', '0424', '0425', '0426', '0427', '0429', '0430', '0431',
       '0432', '0433', '0434', '0435', '0501'],
      dtype='object', name='table_id')

In [36]:
# drop the useless_feature
def process_table_id_vid(df):
    df.drop(["vid"], inplace = True, axis = 1)
    

In [37]:
for col in data_pivot.columns[:50]:
    print ('\n%s属性的不同取值和出现的次数'%col)
    print (data_pivot[col].value_counts())

# 这列先放弃


vid属性的不同取值和出现的次数
96c05ac2aa205f5b503d16f3841ed91f    1
b53c332648bb0f20804173862bf2d9ac    1
213440d484eb90b9abd46c6482e3b39a    1
69396762aa997d5447195aea61188021    1
034f731d58a06e522dbf54c1f6534c99    1
aa7dc7eb79862065b1ae801dda4251a9    1
420e901bb628db112ee5df3bc650d75a    1
3110bb64d9170687524f9013b615e79c    1
43daf57a56a4f61668f7da3674ca5bf9    1
4a729cb8dda97fec40a05e1f3f612c58    1
3340717d02a6b75d84c5856db53d7dd1    1
86768d595351e099e67d0b1f02d2e50c    1
f56e783a0279afa296eccdb999499c6e    1
49a18097b49fbab763c305ee7fb98d0b    1
a01e3368896edc4038bb0b014a12b2ac    1
a846529bbc1fde245b7c00cffe75ad8e    1
c1baef34a0fa682d0541579f5ca5c6fa    1
56c14d3e726e1176781ff072278acb84    1
c09417a3f593f23e1a9ee412628be0aa    1
2d09a99ba93344ad517579093d344a90    1
d4d277a399ec3fc4dfe94d5a2c938695    1
04735b3e256e524c249b81d625282051    1
9b958470650dfb25a94a947e408c8ecb    1
87ddad76a5f63d957aec906455c5508f    1
ea156b360f07e93536a2b4ec5d2e6101    1
c09fa5facb879c4d83b5e6249239b7b8

前列腺大小、形态正常，包膜光滑完整，两侧对称，内部回声均匀。                                         6707
前列腺大小、形态正常，边界清晰，内部回声均匀。                                                1751
前列腺形态大小正常，轮廓光整，实质回声均匀，内可见强回声斑，后方无声影。                                   1608
前列腺形态大小正常，轮廓光整，实质回声均匀，未见明显异常回声。                                         577
前列腺大小、形态正常，包膜光滑完整，内部回声均匀。CDFI：前列腺内未见异常血流信号。                             377
前列腺形态大小正常，轮廓光整，实质回声均匀，内可见强回声斑。                                          190
前列腺形态大小正常，轮廓光整，实质回声欠均匀，内可见强回声斑，后方无声影。                                   153
前列腺大小、形态正常，包膜光滑完整，两侧对称，内部回声欠均匀。                                         112
前列腺形态大小正常，包膜连续，实质回声均匀，内外腺比例正常                                            97
前列腺形态大小正常，轮廓光整，实质回声均匀，内可见强回声斑，后方有声影。                                     77
前列腺形态大小正常，轮廓光整，实质回声不均匀，内可见强回声斑，后方无声影。                                    73
前列腺大小形态尚规则，实质回声均质。                                                       72
前列腺大小、形态正常，包膜完整，内部回声均匀。                                                  69
前列腺大小、形态未见明显

未见异常                     19908
未触及                       4186
肋下未触及                     1171
肝肿大                          9
不大                           5
未及                           4
可触及右肋下1.0cm 剑突下4.0cm         4
可触及左肋下1cm 剑突下1cm             3
可触及右肋下1.0cm 剑突下3.0cm         3
可触及右肋下3.0cm 剑突下6.0cm         2
可触及右肋下1.5cm 剑突下2cm           2
可触及右肋下2.0cm 剑突下4.0cm         2
可触及右肋下2.0cm 剑突下5.0cm         1
肋下约1cm                       1
可触及右肋下0.5cm 剑突下3.0cm         1
腹壁厚触诊不满意                     1
可触及右肋下2cm 剑突下2cm             1
肝肿大, 可触及右肋下7cm 剑突下5cm        1
可触及右肋下1.5cm 剑突下2.5cm         1
有移动性浊音                       1
可触及右肋下1cm 剑突下1.5cm           1
Name: 0406, dtype: int64

0407属性的不同取值和出现的次数
未见异常            22562
未触及              4926
脾脏切除               10
可触及                 7
不大                  3
脾肿大                 3
未 触及                1
脾脏可触及肋下2.5cm        1
未及                  1
弃查                  1
腹壁厚触诊不满意            1
 未触及                1
Name: 0407, dtype: int64

0409属性的不同取值和出现的

未见异常                               24769
正常                                  2155
无                                   2050
各瓣膜听诊区未闻及杂音                         1149
未闻及病理性杂音                            1026
未闻及器质性杂音                             438
二尖瓣区收缩期杂音Ⅱ级                           39
未闻及                                   33
二尖瓣区收缩期杂音                             31
二尖瓣区收缩期杂音Ⅲ级                           14
二尖瓣区三尖瓣区收缩期杂音                         13
二尖瓣区主动脉瓣区收缩期杂音Ⅱ级                       9
二尖瓣区舒张期杂音                              9
主动脉瓣区收缩期杂音                             8
三尖瓣区收缩期杂音                              6
肺动脉瓣区收缩期杂音Ⅱ级                           6
主动脉瓣第二听诊区收缩期杂音Ⅱ级                       6
主动脉瓣第二听诊区舒张期杂音                         4
主动脉瓣区收缩期杂音Ⅱ级                           4
三尖瓣区收缩期杂音Ⅱ级                            4
二尖瓣区主动脉瓣第二听诊区收缩期杂音                     3
二尖瓣区收缩期杂音Ⅰ级                            3
主动脉瓣第二听诊区舒张期杂音,建议进一步检查                 3
二尖瓣区三尖瓣区收缩期杂音Ⅲ级                        3
二尖瓣区收缩期杂音2级     

In [47]:
#综合结果
# gan
# dan
# yi
# pi
# zuoshen
# youshen
# jiazhuangxian
# ruxian
# pangguang
# qianliexian
# rufang
# zigong
# zuofujian
# youfujian
# 
# 0表示正常/1 表示异常
feature_map = {
    "0113": "肝", #肝
    "0114": "胆", #胆
    "0115": "胰", #胰
    "0116": "脾", #脾脏
    "0117": "左肾",
    "0118": "右肾",
    "0119": "膀胱",
    "0120": "前列腺",
    "0121": "子宫",
    "0122": "左附件",
    "0123": "右附件",
    "0124": "输尿管",
    "0101": "甲状腺|心脏|乳腺|动脉",
}
import re
def process_caichao_gan_feature(sum_up, detail):       
    # 缺失值先保留缺失
    if pd.isnull(sum_up) and pd.isnull(detail):
        return np.nan
    if not pd.isnull(sum_up) and not pd.isnull(detail):    
        if "肝" not in sum_up and "肝" not in detail:
            return np.nan
    if not pd.isnull(sum_up):
        if "肝:脂肪肝" in sum_up:
            return 1
        if "肝:肝内钙化" in sum_up:
            return 1
        if  "肝:肝囊肿" in sum_up:
            return 1
        if "未发现明显异常" in sum_up and "肝" in sum_up:
            return 0
    else:
        if not pd.isnull(detail):
            if "未见明显异常" in detail:
                return 0
            if "欠清晰" in detail:
                return 1
            if "回声增强" in detail:
                return 1
            return 0
    return 0
        

data_pivot["gan"] = data_pivot.apply(lambda x: process_caichao_gan_feature(x["0102"], x["0113"]),axis = 1)
print (data_pivot["gan"])

0        0.0
1        0.0
2        1.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       NaN
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       1.0
28       0.0
29       0.0
        ... 
38169    0.0
38170    1.0
38171    1.0
38172    0.0
38173    0.0
38174    1.0
38175    0.0
38176    0.0
38177    0.0
38178    0.0
38179    0.0
38180    0.0
38181    0.0
38182    0.0
38183    0.0
38184    0.0
38185    0.0
38186    1.0
38187    0.0
38188    1.0
38189    1.0
38190    0.0
38191    0.0
38192    0.0
38193    0.0
38194    0.0
38195    0.0
38196    0.0
38197    0.0
38198    0.0
Name: gan, Length: 38199, dtype: float64


In [48]:
def process_caichao_dan_feature(sum_up, detail):       
    # 缺失值先保留缺失
    if pd.isnull(sum_up) and pd.isnull(detail):
        return np.nan
    if not pd.isnull(sum_up) and not pd.isnull(detail):    
        if "胆" not in sum_up and "胆" not in detail:
            return np.nan
    if not pd.isnull(sum_up):
        if "胆:胆囊壁毛糙增厚" in sum_up:
            return 1
        if "胆囊壁毛糙胆囊壁增" in sum_up:
            return 1
        if  "胆:胆囊结石" in sum_up:
            return 1
        if "胆囊未探及" in sum_up:
            return 1
        if "胆:胆囊壁胆固醇结晶" in sum_up:
            return 1
        if "胆囊息肉" in sum_up:
            return 1
        if "胆:胆囊壁增厚" in sum_up:
            return 1
        if "胆囊息肉" in sum_up:
            return 1
        if "未发现明显异常" in sum_up and "胆" in sum_up:
            return 0
    else:
        if not pd.isnull(detail):
            if "未见明显异常" in detail:
                return 0
            if "欠清晰" in detail:
                return 1
            if "回声增强" in detail:
                return 1
            return 0
    return 0
        


In [63]:
def process_caichao_one_feature(sum_up, detail, key_word):       
    # 缺失值先保留缺失
    if pd.isnull(sum_up) and pd.isnull(detail):
        return sum_up
    if not pd.isnull(sum_up) and not pd.isnull(detail):
        if key_word == "心脏":
            if key_word not in sum_up and "房" not in detail:
                return np.nan
        if key_word not in sum_up and key_word not in detail:
            return np.nan
    if not pd.isnull(sum_up):
        if key_word == "脾":
            if "脾脏未探及（自述已切除）" in sum_up:
                return 1
        if key_word == "左肾":
            if "左肾结晶" in sum_up or "左肾钙化" in sum_up or "左肾囊肿" in sum_up:
                return 1
        if key_word == "右肾":
            if "右肾结晶" in sum_up or "右肾钙化" in sum_up or "右肾囊肿" in sum_up or "右肾内钙化" in sum_up:
                return 1
        if key_word == "前列腺":
            if "前列腺钙化灶" in sum_up or "前列腺稍大" in sum_up or "前列腺囊肿" in sum_up:
                return 1
        if key_word == "子宫":
            if "子宫次全切除术后" in sum_up or "子宫体积增大宫颈钙化灶" in sum_up or "宫颈囊肿" in sum_up or "子宫肌瘤" in sum_up:
                return 1
        if key_word == "左附件":
            if "左附件显示不清" in sum_up or "左侧卵巢显示不清" in sum_up:
                return 1
        if key_word == "右附件":
            if "右附件显示不清" in sum_up or "右侧卵巢显示不清" in sum_up:
                return 1
        if key_word == "乳腺":
            if "增生" in sum_up or "囊肿" in sum_up or "结节" in sum_up:
                return 1
        if key_word == "动脉":
            if "硬化" in sum_up:
                return 1
        if "未发现明显异常" in sum_up and key_word in sum_up:
            return 0
        
    else:
        if not pd.isnull(detail) and key_word in detail:
            if "囊性结节" in detail:
                return 1
            if "未见" in detail:
                return 0
            if "无扩张" in detail:
                return 0
            if "无明显" in detail:
                return 0
            if "低回声区" in detail:
                return 1
            if "回声增强" in detail:
                return 1
            if "强回声" in detail:
                return 1
            if "欠佳" in detail:
                return 1
            if "膀胱结石术后" in detail:
                return 1
            
    return 0
        



In [49]:
# table_id = ["0201","0202","0203","0206","0207","0208","0209","0210","0212","0215","0216","0217","0222","0403", "0405", "0407","0409", "0413","0420","0422", "0423"，“0426”，“0427”， “0429”，“0430”，“0431”，“0432”， “0433”， “0434”, "0435", "0501"]
# 其实 0222 这个也是综合检查结果
# 0409 是内科检查结果及病史，得看下是否要做精细
# 0434 也是病史 感觉要细分
def process_table_id_2xx_4xx(x):       
    # 缺失值先保留缺失
    if pd.isnull(x):
        return x
    if "弃查" in x or "exit" in x :
        return np.nan
    if "正常" in x or "未" in x or "无" in x or "不大" in x  or "健康" in x:
        return 0
    else:
        return 1
      

In [56]:
def process_table_id_0421(x):       
    #
    if pd.isnull(x):
        return np.nan
    if "早" in x or "不" in x:
        return 1
    else:
        return 0

In [57]:
# 里面有数字这种
# 看起来是心率
def process_table_id_0424(x):       
    # 缺失值先保留缺失
    # 缺失/过速2/过缓1/正常0
    if pd.isnull(x) or "未查" in x:
        return np.nan
    if "过速" in x:
        return 2
    if "过缓" in x:
        return 1
    if "未" in x or "正常" in x:
        return 0
    temp = ""
    for le in x:
        if le.isdigit():
            temp += temp
    try:
        temp = float(temp)
    except:
        return np.nan
    if temp < 60.0:
        return 1
    elif temp > 100.0:
        return 2
    else:
        return 0
        

In [68]:
# 里面有数字这种
def process_table_id_0425(x):       
    # 缺失值先保留缺失 
    if pd.isnull(x) or "未查" in x:
        return np.nan
    if "正常" in x or "未" in x or "无" in x:
        return 0
    if "粗糙" in x or "缓慢" in x or "急促" in x:
        return 1
    return 0


In [87]:
def fe_first_50(data_pivot):
    # caicao 
    data_pivot["gan"] = data_pivot.apply(lambda x: process_caichao_gan_feature(x["0102"], x["0113"]),axis = 1)
    data_pivot["dan"] = data_pivot.apply(lambda x: process_caichao_dan_feature(x["0102"], x["0114"]),axis = 1)
    data_pivot["yi"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0115"], "胰"),axis = 1)
    data_pivot["pi"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0116"], "脾"),axis = 1)
    data_pivot["zuoshen"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0117"], "左肾"),axis = 1)
    data_pivot["youshen"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0118"], "右肾"),axis = 1)
    data_pivot["pangguang"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0119"], "膀胱"),axis = 1)
    data_pivot["qianliexian"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0120"], "前例腺"),axis = 1)
    data_pivot["zigong"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0121"], "子宫"),axis = 1)
    data_pivot["zuofujian"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0122"], "左附件"),axis = 1)
    data_pivot["youfujian"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0123"], "右附件"),axis = 1)
    data_pivot["youfujian"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0124"], "输尿管"),axis = 1)
    data_pivot["jiazhuangxian"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0101"], "甲状腺"),axis = 1)
    data_pivot["ruxian"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0101"], "乳腺"),axis = 1)
    data_pivot["dongmai"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0101"], "动脉"),axis = 1)
    data_pivot["xinzang"] = data_pivot.apply(lambda x: process_caichao_one_feature(x["0102"], x["0101"], "心脏"),axis = 1)
    #
    # table_id_to_drop
    droped_id = ["vid", "0102"]
    droped_id.extend(feature_map.keys())
    data_pivot.drop(droped_id, inplace = True, axis = 1)
    
    #
    table_id = ["0201","0202","0203","0206","0207","0208","0209","0210","0212","0215","0216","0217","0222","0403", "0405", "0406","0407","0409", "0413","0420","0422", "0423","0426","0427", "0429","0430","0431","0432","0433","0434", "0435", "0501"]
    for t_id in table_id:
       data_pivot[t_id]= data_pivot[t_id].map(process_table_id_2xx_4xx) 
    #
    data_pivot["0421"] = data_pivot["0421"].map(process_table_id_0421)
    data_pivot["0424"] = data_pivot["0424"].map(process_table_id_0424)
    data_pivot["0425"] = data_pivot["0425"].map(process_table_id_0425)
    return data_pivot

feofMy = ['vid', '0101', '0102', '0113', '0114', '0115', '0116', '0117', '0118',
       '0119', '0120', '0121', '0122', '0123', '0124', '0201', '0202', '0203',
       '0206', '0207', '0208', '0209', '0210', '0212', '0215', '0216', '0217',
       '0222', '0403', '0405', '0406', '0407', '0409', '0413', '0420', '0421',
       '0422', '0423', '0424', '0425', '0426', '0427', '0429', '0430', '0431',
       '0432', '0433', '0434', '0435', '0501']
train = data_pivot[feofMy]
train = fe_first_50(train)
train.head()



/Users/baidu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/baidu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/baidu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

table_id,0201,0202,0203,0206,0207,0208,0209,0210,0212,0215,...,youshen,pangguang,qianliexian,zigong,zuofujian,youfujian,jiazhuangxian,ruxian,dongmai,xinzang
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN
3,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,NaN,NaN
4,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,NaN,NaN


In [88]:
train.columns

Index(['0201', '0202', '0203', '0206', '0207', '0208', '0209', '0210', '0212',
       '0215', '0216', '0217', '0222', '0403', '0405', '0406', '0407', '0409',
       '0413', '0420', '0421', '0422', '0423', '0424', '0425', '0426', '0427',
       '0429', '0430', '0431', '0432', '0433', '0434', '0435', '0501', 'gan',
       'dan', 'yi', 'pi', 'zuoshen', 'youshen', 'pangguang', 'qianliexian',
       'zigong', 'zuofujian', 'youfujian', 'jiazhuangxian', 'ruxian',
       'dongmai', 'xinzang'],
      dtype='object', name='table_id')

In [89]:
for col in train.columns[:50]:
    print ('\n%s属性的不同取值和出现的次数'%col)
    print (train[col].value_counts())


0201属性的不同取值和出现的次数
0.0    21588
1.0       86
Name: 0201, dtype: int64

0202属性的不同取值和出现的次数
0.0    21373
1.0      586
Name: 0202, dtype: int64

0203属性的不同取值和出现的次数
0.0    21137
1.0      794
Name: 0203, dtype: int64

0206属性的不同取值和出现的次数
0.0    18880
1.0        4
Name: 0206, dtype: int64

0207属性的不同取值和出现的次数
0.0    20983
1.0       27
Name: 0207, dtype: int64

0208属性的不同取值和出现的次数
0.0    19928
1.0     1271
Name: 0208, dtype: int64

0209属性的不同取值和出现的次数
0.0    18782
1.0     1330
Name: 0209, dtype: int64

0210属性的不同取值和出现的次数
0.0    19877
1.0      964
Name: 0210, dtype: int64

0212属性的不同取值和出现的次数
0.0    20766
1.0        3
Name: 0212, dtype: int64

0215属性的不同取值和出现的次数
0.0    14371
1.0     5777
Name: 0215, dtype: int64

0216属性的不同取值和出现的次数
0.0    20298
1.0      232
Name: 0216, dtype: int64

0217属性的不同取值和出现的次数
0.0    21039
1.0      913
Name: 0217, dtype: int64

0222属性的不同取值和出现的次数
0.0    15486
1.0     6489
Name: 0222, dtype: int64

0403属性的不同取值和出现的次数
0.0    13883
1.0        2
Name: 0403, dtype: int64

0405属性的不同取值和出现的次数
0